In [1]:
from numpy import array
import pandas as pd 
import numpy as np
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Embedding

C:\Users\aishw\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv("data_v2_1096.csv")
docs = []
labels = np.empty(0, float)
#iterate over every row of the dataframe
for index,row in df.iterrows():
  docs.append(row["sent_text"])
  score_v = row["rule_5"]
  labels = np.append(labels, np.array([score_v]), axis = 0)

In [3]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [4]:
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
print(docs)

[[283, 1, 83, 1440, 1, 1441, 2, 215, 1442, 2275, 1, 2276, 164, 14, 526], [2277, 52, 1, 93, 2278, 2, 112, 2279, 84, 17, 6, 2280, 113, 2, 1443, 2, 86, 4, 1444, 1031, 3, 1, 2281, 2, 1, 200], [36, 789, 49, 9, 1, 1445, 1446, 4, 1, 147, 447, 2, 86, 1447, 1, 255, 113, 2, 2282, 790, 4, 329, 2283, 5, 1, 527, 1441, 8, 791, 159, 1, 528, 10, 330, 49, 9, 180, 1032, 331, 86, 4, 637, 1033], [1034, 332, 36, 1443, 27, 2284, 2285, 1, 448, 331, 2, 256, 5, 6, 180, 164], [638, 639, 449, 6, 386, 2286, 45, 1448, 9, 6, 1035, 2, 450, 4, 9, 1, 2287, 125, 2, 333, 17, 6, 334, 2288, 2, 1, 86, 2, 792, 4, 165, 97, 793, 2289], [794, 2290, 14, 147, 165, 97, 1036, 14, 1, 257, 2, 333, 2291, 1, 2292, 2, 331, 86, 5, 126], [28, 1449, 1450, 1, 2293, 2, 1, 83, 529, 10, 331, 86, 9, 127, 22, 4, 1, 792, 8, 2294, 126, 27, 640, 20, 1, 147, 258, 2, 97], [5, 181, 1, 83, 258, 2, 331, 86, 7, 6, 451, 259, 2, 147, 1451, 1037, 4, 2295, 2296, 182, 3, 1, 2297, 795], [1, 641, 452, 134, 81, 216, 1452, 1038, 2298, 2299, 2300, 20, 1, 453, 642

In [5]:
maxLength = [10,110,600]
for lngth in maxLength:
    max_length = lngth
    padded_docs = pad_sequences(encoded_docs, maxlen=lngth, padding = 'pre')
    print(padded_docs)
# load the whole embedding into memory
    embeddings_index = dict()

[[1441    2  215 ...  164   14  526]
 [  86    4 1444 ...    2    1  200]
 [ 330   49    9 ...    4  637 1033]
 ...
 [ 503    3  755 ...    2  316  891]
 [   4 5277  182 ...    1 2274 2146]
 [  17 1555  755 ...   27  401  231]]
[[   0    0    0 ...  164   14  526]
 [   0    0    0 ...    2    1  200]
 [   0    0    0 ...    4  637 1033]
 ...
 [   0    0    0 ...    2  316  891]
 [   0    0    0 ...    1 2274 2146]
 [   0    0    0 ...   27  401  231]]
[[   0    0    0 ...  164   14  526]
 [   0    0    0 ...    2    1  200]
 [   0    0    0 ...    4  637 1033]
 ...
 [   0    0    0 ...    2  316  891]
 [   0    0    0 ...    1 2274 2146]
 [   0    0    0 ...   27  401  231]]


In [6]:
    gd = [50, 100, 200, 300]
    #gd = [100]
    f1 = ['glove.6B.' + str(x) + 'd.txt' for x in gd]
    for fnm in f1:
        print(fnm)

glove.6B.100d.txt


In [7]:
    f = open(fnm, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
#Weight Matrix
    embedding_matrix = zeros((vocab_size, 100))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [ ]:
 l = (('sgd','mean_absolute_error'),
          ('adam','binary_crossentropy'))
    m = [-1,64,128,256,512]
    for a,b in l:
        for val in m:
                model = Sequential()
                e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
                model.add(e)
                if val == -1:
                    model.add(Flatten())
                else:
                    model.add(LSTM(val))
                model.add(Dense(1, activation='sigmoid'))
                model.compile(optimizer=a, loss=b, metrics=['acc'])
                print(model.summary())
                model.fit(padded_docs, labels, epochs=100, verbose=0)
                loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
                print('Accuracy: %f' % (accuracy*100))